# Labelme关键点检测标注转COCO格式

同济子豪兄 呕心沥血写成

2023-3-9

2023-3-10

## 导入工具包

In [1]:
import os
import json

import numpy as np

## 进入labelme标注文件目录

In [2]:
labelme_folder = 'crab-new'

In [3]:
os.chdir(labelme_folder)

In [4]:
os.listdir()

['3.webp',
 '2.webp',
 '5.webp',
 '1.webp',
 '4.webp',
 '2.json',
 '3.json',
 '5.json',
 '4.json',
 '.ipynb_checkpoints',
 '1.json']

## 示例-导入一个labelme格式的json标注文件

In [5]:
# with open('1.json', 'r', encoding='utf-8') as f:
#     labelme = json.load(f)

In [6]:
# labelme['shapes']

In [7]:
# labelme.keys()

In [8]:
# labelme['version']

In [9]:
# labelme['flags']

In [10]:
# 图像文件名
# labelme['imagePath']

In [11]:
# labelme['imageData']

In [12]:
# 图像高度
# labelme['imageHeight']

In [13]:
# 图像宽度
# labelme['imageWidth']

## 创建coco格式的字典

In [14]:
coco = {}

## info

In [15]:
# coco['info'] = {}
# coco['info']['description'] = 'Labelme2coco keypoint format script from Zihao'
# coco['info']['year'] = 2023
# coco['info']['date_created'] = '2023/03/09'

## categories

supercategory：框的类别（一般只有一个类别）

keypoints：节点类别排序，后续的关键点坐标展示顺序

skeleton：可有可无，可视化时需要

In [16]:
class_list = {'supercategory': 'crab',
              'id': 1,
              'name': 'crab',
              'keypoints': ['Left-Eye', 'Right-Eye', 'Left-Sharp', 'Right-Sharp'],
              'skeleton':[[0,2], [1,3]]
             }

In [17]:
coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

## 函数-处理单个labelme标注json文件

In [18]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: # 遍历该json文件中的所有标注

        if each_ann['shape_type'] == 'rectangle': # 筛选出个体框

            # 个体框元数据
            bbox_dict = {}
            bbox_dict['category_id'] = 1
            bbox_dict['segmentation'] = []
            bbox_dict['area'] = 100
            bbox_dict['iscrowd'] = 0
            bbox_dict['segmentation'] = []
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            # print(ANN_ID)
            ANN_ID += 1

            # 获取个体框坐标
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            
            # 筛选出分割多段线
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'polygon': # 筛选出分割多段线标注
                    # 第一个点的坐标
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                        # bbox_dict['segmentation'] = each_ann['points']

            # 筛选出该个体框中的所有关键点
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点横纵坐标
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_left_top_x) & (x<bbox_right_bottom_x) & (y<bbox_right_bottom_y) & (y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
                        
            bbox_dict['num_keypoints'] = len(bbox_keypoints_dict)
            # print(bbox_keypoints_dict)

            # 把关键点按照类别顺序排好
            bbox_dict['keypoints'] = []
            for each_class in class_list['keypoints']:
                if each_class in bbox_keypoints_dict:
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][0])
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][1])
                    bbox_dict['keypoints'].append(2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

## 测试一下函数的效果，处理单个labelme格式的json标注文件

In [19]:
with open('1.json', 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [20]:
process_single_json(labelme)

[{'category_id': 1,
  'segmentation': [[80.0, 203.0],
   [125.88, 151.24],
   [161.18, 135.94],
   [203.53, 118.29],
   [261.18, 127.71],
   [311.76, 153.59],
   [334.12, 180.65],
   [370.59, 187.71],
   [328.24, 197.12],
   [283.53, 227.71],
   [274.12, 250.06],
   [256.47, 272.41],
   [210.59, 281.82],
   [163.53, 288.88],
   [138.82, 263.0],
   [109.41, 245.35],
   [43.53, 238.29]],
  'area': 100,
  'iscrowd': 0,
  'image_id': 1,
  'id': 0,
  'bbox': [11, 112, 377, 220],
  'num_keypoints': 4,
  'keypoints': [176, 123, 2, 224, 117, 2, 55, 236, 2, 353, 186, 2]},
 {'category_id': 1,
  'segmentation': [[75.29, 445.35],
   [129.41, 428.88],
   [152.94, 394.76],
   [162.35, 371.24],
   [176.47, 341.82],
   [183.53, 330.06],
   [243.53, 317.12],
   [284.71, 299.47],
   [328.24, 303.0],
   [362.35, 327.71],
   [401.18, 337.12],
   [447.06, 330.06],
   [418.82, 358.29],
   [401.18, 400.65],
   [387.06, 433.59],
   [356.47, 467.71],
   [323.53, 492.41],
   [265.88, 519.47],
   [211.76, 520.65

## images和annotations

In [21]:
IMG_ID = 0
ANN_ID = 0

In [22]:
# 遍历所有 labelme 格式的 json 文件
for labelme_json in os.listdir(): 
    
    if labelme_json.split('.')[-1] == 'json':
        
        with open(labelme_json, 'r', encoding='utf-8') as f:
            
            labelme = json.load(f)
            
            ## 提取图像元数据
            img_dict = {}
            img_dict['file_name'] = labelme['imagePath']
            img_dict['height'] = labelme['imageHeight']
            img_dict['width'] = labelme['imageWidth']
            img_dict['id'] = IMG_ID
            coco['images'].append(img_dict)
            
            ## 提取框和关键点信息
            coco_annotations = process_single_json(labelme, image_id=IMG_ID)
            coco['annotations'] += coco_annotations
            
            IMG_ID += 1
            
            print(labelme_json, '已处理完毕')

    else:
        pass
    
with open('../labelme2coco.json', 'w') as f:
    json.dump(coco, f, indent=2)

2.json 已处理完毕
3.json 已处理完毕
5.json 已处理完毕
4.json 已处理完毕
1.json 已处理完毕


In [23]:
coco

{'categories': [{'supercategory': 'crab',
   'id': 1,
   'name': 'crab',
   'keypoints': ['Left-Eye', 'Right-Eye', 'Left-Sharp', 'Right-Sharp'],
   'skeleton': [[0, 2], [1, 3]]}],
 'images': [{'file_name': '2.webp', 'height': 343, 'width': 500, 'id': 0},
  {'file_name': '3.webp', 'height': 270, 'width': 400, 'id': 1},
  {'file_name': '5.webp', 'height': 427, 'width': 641, 'id': 2},
  {'file_name': '4.webp', 'height': 500, 'width': 667, 'id': 3},
  {'file_name': '1.webp', 'height': 666, 'width': 500, 'id': 4}],
 'annotations': [{'category_id': 1,
   'segmentation': [],
   'area': 100,
   'iscrowd': 0,
   'image_id': 0,
   'id': 0,
   'bbox': [10, 2, 479, 334],
   'num_keypoints': 4,
   'keypoints': [218, 112, 2, 283, 111, 2, 102, 188, 2, 393, 188, 2]},
  {'category_id': 1,
   'segmentation': [],
   'area': 100,
   'iscrowd': 0,
   'image_id': 1,
   'id': 1,
   'bbox': [2, 13, 390, 249],
   'num_keypoints': 4,
   'keypoints': [162, 91, 2, 228, 92, 2, 75, 154, 2, 321, 155, 2]},
  {'catego

## 验证MS COCO格式的标注

In [38]:
!pip install pycocotools

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp37-cp37m-linux_x86_64.whl size=373954 sha256=ec57289d32ad993c8cc7551fdde48fb56c8fc59ec7b2740b54ffe7752012e781
  Stored in directory: /home/featurize/.cache/pip/wheels/f8/94/70/046149e666bd5812b7de6b87a28dcef238f7162f4108e0b3d8
Successfully built pycocotools


In [22]:
from pycocotools.coco import COCO

my_coco = COCO('../labelme2coco.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
